In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import os
import regex as re

In [2]:
image_data = []
counter = 0

# set the path to the folder containing the images
# path = "images/"

# loop over all the files in the folder
for filename in os.listdir("UTKFace_bw"):    
    # open the image file and convert it to a NumPy array
    if (filename == ".DS_Store" or filename == ".ipynb_checkpoints"):
        continue
    with Image.open(os.path.join("UTKFace_bw", filename)) as img:
    # img_array = np.asarray(img)
        
        parts = filename.split("_") 
        age = int(parts[0])
        gender = int(parts[1])
        ethnicity = int(parts[2])

        image_data.append([filename, img, age, gender, ethnicity])
        counter += 1
        if counter % 5000 == 0:
            print(counter)

# create a DataFrame from the image data
image_df = pd.DataFrame(image_data, columns=["filename", "image", "age", "gender", "ethnicity"])
image_df

5000
10000
15000
20000
25000
30000
35000
40000
45000


,filename,image,age,gender,ethnicity
0,9_1_2_20161219204347420.jpg.chip.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,9,1,2
1,1_0_2_20161219195848259.jpg.chip.jpgbw.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1,0,2
2,36_0_1_20170117163203851.jpg.chip.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,36,0,1
3,86_1_0_20170120225751953.jpg.chip.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,86,1,0
4,1_1_0_20170110212647258.jpg.chip.jpgbw.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1,1,0
...,...,...,...,...,...
47411,4_1_3_20161219230102512.jpg.chip.jpgbw.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,4,1,3
47412,86_0_2_20170112191332213.jpg.chip.jpgbw.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,86,0,2
47413,8_0_0_20170110215642859.jpg.chip.jpgbw.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8,0,0
47414,28_0_1_20170117015458481.jpg.chip.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,28,0,1


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [4]:
from torch.utils.data import Dataset, DataLoader, random_split


class ImageDataFrameDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.root_dir, row['filename'])
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)
        
        label = row['gender']
        # print(label)
        
        return image, label

In [5]:
image_dataset = ImageDataFrameDataset(
    dataframe=image_df,
    root_dir='UTKFace_bw/',
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)
data_loader = DataLoader(
    image_dataset, batch_size=32, shuffle=True, num_workers=4
)

train_ratio = 0.8

# Calculate the sizes of the train and test sets based on the split ratio
train_size = int(len(image_dataset) * train_ratio)
test_size = len(image_dataset) - train_size

# Use random_split to split the dataset into train and test sets

train_dataset, test_dataset = random_split(image_dataset, [train_size, test_size])

In [ ]:
import time

# Load the ResNet18 model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Create data loaders for the train and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train the model
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

start_time = time.time()

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set after each epoch
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Elapsed time: {time.time() - start_time:.4f} seconds")
    print(f'Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

/Users/avacrnkovic-rubsamen/opt/anaconda3/envs/cos429/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/Users/avacrnkovic-rubsamen/opt/anaconda3/envs/cos429/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Elapsed time: 15609.7146 seconds
Epoch 0: Test Loss: 0.0105, Accuracy: 85.18%
Elapsed time: 27061.7021 seconds
Epoch 1: Test Loss: 0.0087, Accuracy: 87.99%
Elapsed time: 34584.7617 seconds
Epoch 2: Test Loss: 0.0080, Accuracy: 88.88%
Elapsed time: 42479.0955 seconds
Epoch 3: Test Loss: 0.0075, Accuracy: 90.13%
Elapsed time: 49958.9497 seconds
Epoch 4: Test Loss: 0.0071, Accuracy: 90.68%
Elapsed time: 57403.6238 seconds
Epoch 5: Test Loss: 0.0068, Accuracy: 91.04%
Elapsed time: 74045.7584 seconds
Epoch 6: Test Loss: 0.0067, Accuracy: 91.59%
